## Feature Engineering

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv("data_revised.csv")


In [2]:
#Lets start by defining prediction problem: Identifying customers that are going to invest.
#This means among customers who are not already investing, who is mostly likely to invest.
#To Reduce scope, we can sort out customers who where investing at the start 2022 (the year of our data). 
#If I had more historical data, all customers would potentially be eligible. But since "investing for the first time" is a one time event for all customers
#We sort out customers out as soon as they have invested something. 
#Lets start by defining already invested customers to sort them out. 
#Normally I would define already invested customers as customers having Investment_Assets>0 at any time t-n, where n=>0. But since we dont have unlimited history,
#I define it as Investment_Assets>0 in the first month of 2022 or if the customers are new in 2022, their first month.

#Reduce dataset to include only the first observation of every customer

df = df.copy()
df['FactDt'] = pd.to_datetime(df['FactDt'])

df_first_obs = (
    df.sort_values(['Customer_number', 'FactDt'])
      .drop_duplicates(subset=['Customer_number'], keep='first')
      .copy()
)

df_first_obs.loc[((df_first_obs['Investment_Assets']>0)|(df_first_obs['AvgInvestmentsValueDKK']>0)), "Already_invested"]=True

print('No of customers already investing: ', df_first_obs['Already_invested'].sum())

#Sort out those customers (pi=potential investors)
df_pi = df.loc[~df['Customer_number'].isin(df_first_obs.loc[df_first_obs['Already_invested']==True, 'Customer_number'])]

print('No of potential investors: ', df_pi['Customer_number'].nunique())


No of customers already investing:  3316
No of potential investors:  7135


In [3]:
#Next i also want to reduced the potential investors, by sorting out "kids" - it seems unethical to do any investing marketing towards them + the marketing if it were should be targeting the parents to invest "Børneopsparingen". 
# So i reduce the scope to customers above 16 years old:
df_pi = df_pi.loc[df_pi['CustAgeInYears']>16]

print('No of potential investors: ', df_pi['Customer_number'].nunique())

No of potential investors:  6252


In [4]:
# Define target: first-time invest event
df_pi = df_pi.sort_values(['Customer_number', 'FactDt']).copy()

# Invested at time t (using Investment_Assets)
invested_t = df_pi['Investment_Assets'] > 0

# Had invested at any previous time t-n, n>0
had_invested_before = invested_t.groupby(df_pi['Customer_number']).cummax().groupby(df_pi['Customer_number']).shift(fill_value=False)

# First-time invest: invested now, but never invested before
df_pi['first_time_invest'] = invested_t & (~had_invested_before)

print('First-time invest events:', int(df_pi['first_time_invest'].sum()))

#I want the flag moved up one time (so the predvious month is flagged as first time investors. Cause it is these once I want to predict - just before they actaully invest)
#But also to prevent any data leakage.

df_pi['first_time_invest']=df_pi['first_time_invest'].shift(-1)

#Now sort out all later months per customer after a True Value in first_time_invest



First-time invest events: 93


In [5]:
# Force boolean flag (handles NaN safely)
flag = df_pi['first_time_invest'].fillna(False).astype(bool)

# True for rows AFTER the first True within each customer
seen_true_before = (
    flag.groupby(df_pi['Customer_number']).cummax()
        .groupby(df_pi['Customer_number']).shift(fill_value=False)
        .astype(bool)
)

# Keep rows up to and including first True
df_pi = df_pi.loc[~seen_true_before].copy()

In [6]:
#max view pandas columns
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x).replace(',', 'X').replace('.', ',').replace('X', '.'))

In [7]:
#Its time to define the prediction problem even more. Now there is two possible ways to go with it, either is a binary classification problem, where we just want to predict is this a possible first time investor?
#Or as a time to event problem, where we predict and rank customers in orders of who has the least predicted time to event. However in this case, which so "few history", and not a lot of data, this will probably not work on this case. 
#But I will give it a try later. 
#But lets start by doing it is a binary classification problem. And I define "first time invest" customers as customer who gonna invest within the following year. Since we only have year of data, all customers with "first_time_invest" flag will be considered with a target 1 variable


future_investors = set(df_pi.loc[df_pi['first_time_invest'].fillna(False), 'Customer_number'])
df_pi['BI_Label'] = df_pi['Customer_number'].isin(future_investors).astype(int)

In [8]:

#Lets also define target variable if we were to handle the problem as a time to event. Since we have time in months our time to event is gonna be in months.
# Row-based TTE target with censoring at Jan 2023
df_pi = df_pi.sort_values(['Customer_number', 'FactDt']).copy()
df_pi['FactDt'] = pd.to_datetime(df_pi['FactDt'])

today_ref = pd.Timestamp('2023-01-31')  # pretend "today"

def add_tte_row_event(group):
    group = group.sort_values('FactDt').copy().reset_index(drop=True)
    event_mask = group['first_time_invest'].fillna(False).astype(bool).to_numpy()
    event_pos = np.where(event_mask)[0]

    if len(event_pos) > 0:
        # Use first observed event
        e = event_pos[0]
        group['TTE'] = (e - np.arange(len(group)) + 1).astype(float)  # event row = 1
        # optional: remove rows after event
        group.loc[np.arange(len(group)) > e, 'TTE'] = np.nan
        group['event_observed'] = 1
    else:
        # No event: censored at Jan 2023 (calendar-month distance, starts at 1)
        group['TTE'] = (
            (today_ref.year - group['FactDt'].dt.year) * 12
            + (today_ref.month - group['FactDt'].dt.month)
            + 1
        ).astype(float)
        group['TTE'] = group['TTE'].clip(lower=1)
        group['event_observed'] = 0

    return group

df_pi = (
    df_pi.groupby('Customer_number', group_keys=False)
         .apply(add_tte_row_event)
)

# quick check
df_pi[['Customer_number', 'FactDt', 'first_time_invest', 'event_observed', 'TTE']].head(5)

,Customer_number,FactDt,first_time_invest,event_observed,TTE
0,000022D8D733FA51200BF5F45C37AED2,2022-01-31,False,0,"13,00"
1,000022D8D733FA51200BF5F45C37AED2,2022-02-28,False,0,"12,00"
2,000022D8D733FA51200BF5F45C37AED2,2022-03-31,False,0,"11,00"
3,000022D8D733FA51200BF5F45C37AED2,2022-04-30,False,0,"10,00"
4,000022D8D733FA51200BF5F45C37AED2,2022-05-31,False,0,"9,00"


In [9]:
#Now that actually have defined the prediction problems, we are gonna do some nice feature engineering. Now most obvious is to start by converting all "stamdata" features to something useful, and do some feature engineering to capture time trends in the data

#calculate seniority in years as time difference between CustEstbDt and FactDt
df_pi['CustEstbDt'] = pd.to_datetime(df_pi['CustEstbDt'], errors='coerce')
df_pi['anc_y'] = (df_pi['FactDt'] - df_pi['CustEstbDt']).dt.days / 365.25

#drop column CustEstbDt
df_pi = df_pi.drop(columns=['CustEstbDt'])

#Convert CustGenderCd to dummy variable
df_pi = pd.get_dummies(df_pi, columns=['CustGenderCd'], drop_first=True)



In [10]:
# Calculate trend features for Business_Volume, Net_Wealth, Financial_Assets
# using log differences instead of pct_change

def rolling_slope(values):
    y = pd.Series(values).astype(float)
    x = np.arange(len(y), dtype=float)
    mask = y.notna()
    if mask.sum() < 2:
        return np.nan
    return np.polyfit(x[mask], y[mask], 1)[0]

for col in ['Business_Volume', 'Net_Wealth', 'Financial_Assets']:
    # log level (safe for zeros; clip protects against negatives)
    log_col = np.log1p(df_pi[col].clip(lower=0))

    # 1-month log difference
    df_pi[f'{col}_log_diff_1m'] = log_col.groupby(df_pi['Customer_number']).diff(1)

    # optional: convert log-diff to approx % change
    df_pi[f'{col}_log_diff_1m_pct'] = np.expm1(df_pi[f'{col}_log_diff_1m']) * 100

    # rolling 4m trend from slope in log space (already robust)
    slope_log = (
        log_col.groupby(df_pi['Customer_number'])
               .transform(lambda s: s.rolling(window=4, min_periods=2).apply(rolling_slope, raw=True))
    )
    df_pi[f'{col}_trend_4m'] = np.expm1(slope_log) * 100

In [11]:
#Calculate trend in in number of times visiting website and app:

for col in ['CustMobileLogOnPerMth', 'CustNetBankLogOnPerMth']:
    g = df_pi.groupby('Customer_number')[col]

    # 1-month absolute difference
    df_pi[f'{col}_diff_1m'] = g.diff(1)

    # Mean of the last 4 monthly absolute differences
    df_pi[f'{col}_diff_4m_mean'] = (
        df_pi.groupby('Customer_number')[f'{col}_diff_1m']
             .transform(lambda s: s.rolling(window=4, min_periods=1).mean())
    )

In [12]:
#Lastly thinking about what other values could indicate a shift in life. In segmentation we noticed different depending on adult family members and children. So maybe we can also capture changes in the number of adult family members and children in the household as features.
#capture changes in fam_members as two variables, (one as the latest change (has it gone down or up)) and one as the month since the last change (in the last 6 months)
#doing it for fam_members will captue both adult changes and children changes since we will still use how many adult and family members there is

col = 'fam_members'  # change name if your column differs

df_pi = df_pi.sort_values(['Customer_number', 'FactDt']).copy()

# 1) Latest change (up/down/same)
df_pi[f'{col}_diff_1m'] = df_pi.groupby('Customer_number')[col].diff(1)
df_pi[f'{col}_change_dir'] = np.sign(df_pi[f'{col}_diff_1m']).astype('float')
# interpretation: -1 = down, 0 = no change, 1 = up

# 2) Months since last change (looking back max 4 months)
changed = df_pi[f'{col}_diff_1m'].fillna(0).ne(0)

def months_since_last_change(flag_series):
    idx = np.arange(len(flag_series), dtype=float)
    last_change_idx = np.where(flag_series.values, idx, np.nan)
    last_change_idx = pd.Series(last_change_idx).ffill().to_numpy()
    out = idx - last_change_idx
    out[np.isnan(last_change_idx)] = 0
    return pd.Series(out, index=flag_series.index)

df_pi[f'{col}_months_since_change'] = (
    changed.groupby(df_pi['Customer_number'], group_keys=False)
           .apply(months_since_last_change)
)

# Keep only "in last 6 months" (else NaN)
df_pi.loc[df_pi[f'{col}_months_since_change'] > 4, f'{col}_months_since_change'] = 0 #max time


In [13]:
#Lastly to test if features like "take up a house loan in the last 4 months" or "car loan in the last 4 months" could be useful, since these are big life events that could trigger investing.
#I will do it for both house loan and car loan, and I will capture it as
#1) a binary variable indicating if they have taken up the loan in the last 4 months
#2) a variable indicating how many months since they took up the loan (capped at 4 months, else 4) (similar to the fam_members change above)

for loan_col in ['HomeLoan', 'CAR_LOANS']:  
    # 1) Binary variable for taking up the loan in the last 4 months
    df_pi[f'{loan_col}_taken_4m'] = (
        df_pi.groupby('Customer_number')[loan_col]
             .transform(lambda s: s.rolling(window=4, min_periods=1).max())
    )

    # 2) Months since took up the loan (capped at 4)
    taken = df_pi[loan_col].fillna(0).astype(bool)

    def months_since_taken(flag_series):
        idx = np.arange(len(flag_series), dtype=float)
        last_taken_idx = np.where(flag_series.values, idx, np.nan)
        last_taken_idx = pd.Series(last_taken_idx).ffill().to_numpy()
        out = idx - last_taken_idx
        out[np.isnan(last_taken_idx)] = 0
        return pd.Series(out, index=flag_series.index)

    df_pi[f'{loan_col}_months_since_taken'] = (
        taken.groupby(df_pi['Customer_number'], group_keys=False)
             .apply(months_since_taken)
    )

    # Keep only "in last 4 months" (else NaN)
    df_pi.loc[df_pi[f'{loan_col}_months_since_taken'] > 4, f'{loan_col}_months_since_taken'] = 0

In [17]:
#Now columns we actually want to use to train the model:

columns_to_use = [
    'Customer_number', 'FactDt', 'BI_Label', 'event_observed', 'TTE','Business_Volume',
    'Gross_wealth','Net_Wealth', 'House_free_Equity', 'Financial_Assets',
    'RealEstateValue','HomeLoan', 'CAR_LOANS', 'All_Loans',
    'BOAT_LOANS', 'Other_loans', 'CustAgeInYears', 'CustGenderCd_M', 'CustGenderCd_N',
    'fam_members_adult', 'fam_members', 'CustMobileLogOnPerMth',
    'CustNetBankLogOnPerMth', 'IsGeneralMarketingAllowed',
    'IsMarketingForEmailAllowed', 'IsMarketingForSmsAllowed',
    'IsMarketingForLetterAllowed', 'IsMarketingForPhoneAllowed',
    'IsMarketingForEbankAllowed', 'AvgDepositsValueDKK',
    'AvgPensionValueDKK', 'AvgHomeFinanceValueDKK', 'AvgOtherLoansValueDKK',
    'AvgInvestmentsValueDKK', 'Deposits_Cards_lowest',
    'Deposits_ordinary_lowest', 'Creditcard_overdraft_lowest',
    'anc_y',  'Business_Volume_log_diff_1m_pct', 'Business_Volume_trend_4m', 
     'Net_Wealth_log_diff_1m_pct', 'Net_Wealth_trend_4m'
    , 'Financial_Assets_log_diff_1m_pct',
    'Financial_Assets_trend_4m', 'CustMobileLogOnPerMth_diff_1m',
    'CustMobileLogOnPerMth_diff_4m_mean', 'CustNetBankLogOnPerMth_diff_1m',
    'CustNetBankLogOnPerMth_diff_4m_mean', 
     'fam_members_diff_1m', 'fam_members_change_dir',
    'fam_members_months_since_change', 'HomeLoan_taken_4m',
    'HomeLoan_months_since_taken', 'CAR_LOANS_taken_4m',
    'CAR_LOANS_months_since_taken']

df_to_train= df_pi[columns_to_use].copy()

In [18]:
df_to_train

,Customer_number,FactDt,BI_Label,event_observed,TTE,Business_Volume,Gross_wealth,Net_Wealth,House_free_Equity,Financial_Assets,RealEstateValue,HomeLoan,CAR_LOANS,All_Loans,BOAT_LOANS,Other_loans,CustAgeInYears,CustGenderCd_M,CustGenderCd_N,fam_members_adult,fam_members,CustMobileLogOnPerMth,CustNetBankLogOnPerMth,IsGeneralMarketingAllowed,IsMarketingForEmailAllowed,IsMarketingForSmsAllowed,IsMarketingForLetterAllowed,IsMarketingForPhoneAllowed,IsMarketingForEbankAllowed,AvgDepositsValueDKK,AvgPensionValueDKK,AvgHomeFinanceValueDKK,AvgOtherLoansValueDKK,AvgInvestmentsValueDKK,Deposits_Cards_lowest,Deposits_ordinary_lowest,Creditcard_overdraft_lowest,anc_y,Business_Volume_log_diff_1m_pct,Business_Volume_trend_4m,Net_Wealth_log_diff_1m_pct,Net_Wealth_trend_4m,Financial_Assets_log_diff_1m_pct,Financial_Assets_trend_4m,CustMobileLogOnPerMth_diff_1m,CustMobileLogOnPerMth_diff_4m_mean,CustNetBankLogOnPerMth_diff_1m,CustNetBankLogOnPerMth_diff_4m_mean,fam_members_diff_1m,fam_members_change_dir,fam_members_months_since_change,HomeLoan_taken_4m,HomeLoan_months_since_taken,CAR_LOANS_taken_4m,CAR_LOANS_months_since_taken
0,000022D8D733FA51200BF5F45C37AED2,2022-01-31,0,0,"13,00","13,45","13,45","13,45","0,00","13,45","0,00","0,00","0,00","0,00","0,00","0,00","28,00",0,0,2,2,0,0,1,0,0,0,0,0,"13,45","0,00","0,00","0,00",0,"13,45","13,45","0,00","1,26",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,00","0,00","0,00","0,00","0,00"
1,000022D8D733FA51200BF5F45C37AED2,2022-02-28,0,0,"12,00","13,45","13,45","13,45","0,00","13,45","0,00","0,00","0,00","0,00","0,00","0,00","28,00",0,0,2,2,0,0,1,0,0,0,0,0,"13,45","0,00","0,00","0,00",0,"13,45","13,45","0,00","1,34","0,00","-0,00","0,00","-0,00","0,00","-0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,000022D8D733FA51200BF5F45C37AED2,2022-03-31,0,0,"11,00","13,45","13,45","13,45","0,00","13,45","0,00","0,00","0,00","0,00","0,00","0,00","28,00",0,0,2,2,0,0,1,0,0,0,0,0,"13,45","0,00","0,00","0,00",0,"13,45","13,45","0,00","1,43","0,00","-0,00","0,00","-0,00","0,00","-0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,000022D8D733FA51200BF5F45C37AED2,2022-04-30,0,0,"10,00","818,45","818,45","818,45","0,00","818,45","0,00","0,00","0,00","0,00","0,00","0,00","28,00",0,0,2,2,0,0,1,0,0,0,0,0,"630,62","0,00","0,00","0,00",0,"13,45","13,45","0,00","1,51","5.570,93","235,81","5.570,93","235,81","5.570,93","235,81","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,000022D8D733FA51200BF5F45C37AED2,2022-05-31,0,0,"9,00","818,45","818,45","818,45","0,00","818,45","0,00","0,00","0,00","0,00","0,00","0,00","28,00",0,0,2,2,0,0,1,0,0,0,0,0,"818,45","0,00","0,00","0,00",0,"818,45","818,45","0,00","1,59","0,00","402,88","0,00","402,88","0,00","402,88","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,0174CFCBED87171B4F6A651734F836E4,2022-07-31,0,0,"7,00","1.957.353,73","4.364.667,34","2.971.980,95","2.407.313,61","564.667,34","3.800.000,00","-1.392.686,39","0,00","-1.392.686,39","0,00","0,00","61,00",1,0,2,2,0,1,1,0,0,0,0,0,"543.163,49","0,00","-1.467.887,13","0,00",0,"526.440,22","526.440,22","0,00","25,51","1,91","0,04","-1,18","1,20","0,11","0,51","0,00","0,00","0,00","-0,25","0,00","0,00","0,00","-1.356.663,84","0,00","0,00","0,00"
7,0174CFCBED87171B4F6A651734F836E4,2022-08-31,0,0,"6,00","1.871.228,88","4.360.875,31","3.050.521,74","2.489.646,43","560.875,31","3.800.000,00","-1.310.353,57","0,00","-1.310.353,57","0,00","0,00","61,00",1,0,2,2,0,1,1,0,0,0,0,0,"547.337,46","0,00","-1.459.888,70","0,00",0,"529.276,56","529.276,56","0,00","25,60","-4,40","-1,06","2,64","1,26","-0,67","-0,49","0,00","0,00","0,00","-0,25","0,00","0,00","0,00","-1.310.353,57","0,00","0,00","

In [19]:
df_to_train.to_csv("df_to_train.csv", index=False)